# 데이터 준비 코드

In [104]:
import numpy as np
import pandas as pd

In [105]:
# 데이터 불러오기
hotels_predict = pd.read_csv('ML_Data/hotels_predict.csv')
move_predict = pd.read_csv('ML_Data/move_predict.csv')
searches_predict = pd.read_csv('ML_Data/searches_predict.csv')
visitor_predict= pd.read_csv('ML_Data/visitor_predict.csv')
data= pd.read_csv('ML_Data/merged_predict.csv')

# 모델 학습 및 예측

In [106]:
!pip install xgboost
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


학습모델1

In [108]:
# 특성과 타겟 변수 정의
X = data[['숙박업소 개수', '관외 이동량', '검색건수', '일시']]
y = data['방문자수']

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# XGBoost 모델 학습
xgb_model1 = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=5)
xgb_model1.fit(X_train, y_train)

# 모델 평가
y_pred = xgb_model1.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 606951640423.301


In [93]:
#MSE가 클수록 예측 오차가 크다는 것을 의미,
#작을수록 예측 성능이 좋다고 해석함.

#보통 MSE는 실제 값과 예측 값의 차이를 제곱한 값들의 평균이다.

학습모델 2

In [109]:
from sklearn.model_selection import GridSearchCV, train_test_split
import xgboost as xgb
from sklearn.metrics import mean_squared_error

# 데이터 준비
X = data[['숙박업소 개수', '관외 이동량', '검색건수', '일시']]
y = data['방문자수']

# 학습 데이터와 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# XGBoost 모델 정의
model = xgb.XGBRegressor(objective='reg:squarederror')

# 하이퍼파라미터 그리드
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.3]
}

# Grid Search를 사용하여 최적의 하이퍼파라미터 찾기
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X_train, y_train)

# 최적의 모델 출력
print("최적의 하이퍼파라미터:", grid_search.best_params_)

# 최적의 모델로 예측 및 평가
xgb_model2 = grid_search.best_estimator_
y_pred = xgb_model2.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"테스트 세트에서의 MSE: {mse}")

Fitting 3 folds for each of 27 candidates, totalling 81 fits
최적의 하이퍼파라미터: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 300}
테스트 세트에서의 MSE: 520267886092.5014


학습모델 3

In [110]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# 데이터 준비
X = data[['숙박업소 개수', '관외 이동량', '검색건수', '일시']]
y = data['방문자수']

# 학습 데이터와 검증 데이터로 분할
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# XGBoost 모델 정의
xgb_model3 = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=1000, learning_rate=0.1, max_depth=5)

# eval_set 설정
eval_set = [(X_train, y_train), (X_valid, y_valid)]

# 모델 훈련
xgb_model3.fit(X_train, y_train, eval_set=eval_set, verbose=True)

# 최적의 반복 횟수를 자동으로 선택한 모델로 예측 및 평가
y_pred = xgb_model3.predict(X_valid)
mse = mean_squared_error(y_valid, y_pred)
print(f"검증 세트에서의 MSE: {mse}")

[0]	validation_0-rmse:3074403.09411	validation_1-rmse:3017898.57954


[1]	validation_0-rmse:2808542.23623	validation_1-rmse:2761933.58826
[2]	validation_0-rmse:2569837.16336	validation_1-rmse:2531493.40372
[3]	validation_0-rmse:2356995.32350	validation_1-rmse:2327579.24196
[4]	validation_0-rmse:2165358.66657	validation_1-rmse:2142765.66178
[5]	validation_0-rmse:1993593.61060	validation_1-rmse:1981513.49235
[6]	validation_0-rmse:1840383.07463	validation_1-rmse:1836446.06629
[7]	validation_0-rmse:1705463.43481	validation_1-rmse:1709863.49777
[8]	validation_0-rmse:1583560.55271	validation_1-rmse:1599849.52473
[9]	validation_0-rmse:1476445.36937	validation_1-rmse:1500291.14277
[10]	validation_0-rmse:1380323.94223	validation_1-rmse:1410090.26270
[11]	validation_0-rmse:1296932.18721	validation_1-rmse:1335329.46640
[12]	validation_0-rmse:1222999.06346	validation_1-rmse:1271897.34058
[13]	validation_0-rmse:1152698.27386	validation_1-rmse:1211854.33966
[14]	validation_0-rmse:1092909.90536	validation_1-rmse:1163964.24222
[15]	validation_0-rmse:1042452.46542	valida

# 2024년 데이터 생성 및 예측

모델1의 에측

In [118]:
# 2024년 예측 데이터 생성
months_2024 = [1, 2, 3, 4, 5]
future_data = pd.DataFrame()

for month in months_2024:
    for region in data['지역명'].unique():
        # 해당 지역의 2023년 데이터가 없는 경우 예외 처리
        if region not in hotels_predict['지역명'].unique() or region not in move_predict['지역명'].unique() or region not in searches_predict['지역명'].unique():
            continue
        
        hotel_count = hotels_predict.loc[hotels_predict['지역명'] == region, '숙박업소 개수'].values[0]
        
        # 2024년 데이터가 없으므로 과거 데이터의 평균을 사용합니다.
        move_value = move_predict.loc[move_predict['지역명'] == region, '관외 이동량'].mean()
        search_count = searches_predict.loc[searches_predict['지역명'] == region, '검색건수'].mean()
        
        future_data = pd.concat([
            future_data,
            pd.DataFrame({
                '일시': [month],
                '지역명': [region],
                '숙박업소 개수': [hotel_count],
                '관외 이동량': [move_value],
                '검색건수': [search_count],
                'month': [month]
            })
        ], ignore_index=True)

# 결측값 처리 (필요시)
future_data.fillna(0, inplace=True)

# 예측
X_future = future_data[['숙박업소 개수', '관외 이동량', '검색건수', 'month']]
future_data['예측 방문자수'] = xgb_model.predict(X_future)

print(future_data[['일시', '지역명', '예측 방문자수']])

# CSV 파일로 저장
future_data.to_csv('future_data1.csv', index=False)


     일시          지역명      예측 방문자수
0     1  강원특별자치도 춘천시  8613501.000
1     1  강원특별자치도 원주시  9939502.000
2     1  강원특별자치도 강릉시  8068274.000
3     1  강원특별자치도 동해시  1958529.250
4     1  강원특별자치도 태백시  1338308.750
..   ..          ...          ...
815   5     충청북도 증평군  1238881.750
816   5     충청북도 진천군  3180953.000
817   5     충청북도 괴산군  1834421.875
818   5     충청북도 음성군  3591290.500
819   5     충청북도 단양군  1798989.625

[820 rows x 3 columns]


모델 2의 예측

In [121]:
# 2024년 예측 데이터 생성
months_2024 = [1, 2, 3, 4, 5]
future_data = pd.DataFrame()

for month in months_2024:
    for region in data['지역명'].unique():
        # 해당 지역의 2023년 데이터가 없는 경우 예외 처리
        if region not in hotels_predict['지역명'].unique() or region not in move_predict['지역명'].unique() or region not in searches_predict['지역명'].unique():
            continue
        
        hotel_count = hotels_predict.loc[hotels_predict['지역명'] == region, '숙박업소 개수'].values[0]
        
        # 2024년 데이터가 없으므로 과거 데이터의 평균을 사용합니다.
        move_value = move_predict.loc[move_predict['지역명'] == region, '관외 이동량'].mean()
        search_count = searches_predict.loc[searches_predict['지역명'] == region, '검색건수'].mean()
        
        future_data = pd.concat([
            future_data,
            pd.DataFrame({
                '일시': [month],
                '지역명': [region],
                '숙박업소 개수': [hotel_count],
                '관외 이동량': [move_value],
                '검색건수': [search_count],
                'month': [month]
            })
        ], ignore_index=True)

# 결측값 처리 (필요시)
future_data.fillna(0, inplace=True)

# 예측
X_future = future_data[['숙박업소 개수', '관외 이동량', '검색건수', '일시']]
future_data['예측 방문자수'] = xgb_model2.predict(X_future)

print(future_data[['일시', '지역명', '예측 방문자수']])

# CSV 파일로 저장
future_data.to_csv('future_data2.csv', index=False)


     일시          지역명      예측 방문자수
0     1  강원특별자치도 춘천시  8944691.000
1     1  강원특별자치도 원주시  9427674.000
2     1  강원특별자치도 강릉시  8160717.500
3     1  강원특별자치도 동해시  2049472.750
4     1  강원특별자치도 태백시  1275827.875
..   ..          ...          ...
815   5     충청북도 증평군  1350359.000
816   5     충청북도 진천군  3247483.750
817   5     충청북도 괴산군  1800379.500
818   5     충청북도 음성군  3754867.000
819   5     충청북도 단양군  1670933.000

[820 rows x 3 columns]


모델3의 예측

In [123]:
# 2024년 예측 데이터 생성
months_2024 = [1, 2, 3, 4, 5]
future_data = pd.DataFrame()

for month in months_2024:
    for region in data['지역명'].unique():
        # 해당 지역의 2023년 데이터가 없는 경우 예외 처리
        if region not in hotels_predict['지역명'].unique() or region not in move_predict['지역명'].unique() or region not in searches_predict['지역명'].unique():
            continue
        
        hotel_count = hotels_predict.loc[hotels_predict['지역명'] == region, '숙박업소 개수'].values[0]
        
        # 2024년 데이터가 없으므로 과거 데이터의 평균을 사용합니다.
        move_value = move_predict.loc[move_predict['지역명'] == region, '관외 이동량'].mean()
        search_count = searches_predict.loc[searches_predict['지역명'] == region, '검색건수'].mean()
        
        future_data = pd.concat([
            future_data,
            pd.DataFrame({
                '일시': [month],
                '지역명': [region],
                '숙박업소 개수': [hotel_count],
                '관외 이동량': [move_value],
                '검색건수': [search_count],
                'month': [month]
            })
        ], ignore_index=True)

# 결측값 처리 (필요시)
future_data.fillna(0, inplace=True)

# 예측
X_future = future_data[['숙박업소 개수', '관외 이동량', '검색건수', '일시']]
future_data['예측 방문자수'] = xgb_model3.predict(X_future)

print(future_data[['일시', '지역명', '예측 방문자수']])

# CSV 파일로 저장
future_data.to_csv('future_data3.csv', index=False)


     일시          지역명      예측 방문자수
0     1  강원특별자치도 춘천시  9052757.000
1     1  강원특별자치도 원주시  9462811.000
2     1  강원특별자치도 강릉시  8127627.500
3     1  강원특별자치도 동해시  2236159.000
4     1  강원특별자치도 태백시  1279462.875
..   ..          ...          ...
815   5     충청북도 증평군  1390784.750
816   5     충청북도 진천군  3356567.000
817   5     충청북도 괴산군  1801725.500
818   5     충청북도 음성군  3964067.500
819   5     충청북도 단양군  1601859.250

[820 rows x 3 columns]
